# HPC in Python with TensorFlow

In [1]:
try:
    import google.colab
except ImportError:
    pass
else:
    !pip install zfit numba numpy tensorflow_probability

In [2]:
import tensorflow as tf

What we will see (not in the Jupyter Notebook) are a bunch of messages, including the following:
`tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA`

What could they mean?

AVX stands for Advanced Vector Extensions and are instruction that the CPU can perform. They are specializations on vector operations (remember? SIMD, CPU inststruction set, etc.)

Why do they appear?

The code that we are using was not compiled with this flag on. This means, TensorFlow assumes that the CPU does not support this instructions and instead uses non-optimized ones. The reason is that this allows the binary (=compiled code) to also be run on a CPU that does not support then. While we use only some speed.
(yes, technically TensorFlow can be faster when compiled natively on your computer, but then it takes time and effort)

In [3]:
import tensorflow_probability as tfp
import zfit
from zfit import z
import numpy as np
import numba

Let's start with a simple comparison of Numpy, an AOT compiled library, versus pure Python

In [4]:
size1 = 100000
list1 = [np.random.uniform() for _ in range(size1)]
list2 = [np.random.uniform() for _ in range(size1)]
list_zeros = [0] * size1

ar1 = np.array(list1)
ar2 = np.random.uniform(size=size1)  # way more efficient!
ar_zeros = np.zeros_like(ar1) # quite useful function the *_like -> like the object
# we could also create the below, in general better:
# ar_empty = np.empty(shape=size1, dtype=np.float64)

In [5]:
%%timeit
for i in range(size1):
    list_zeros[i] = list1[i] + list2[i]

9.51 ms ± 155 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
ar1 + ar2

41.4 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


( _playground_ : we can also try assignements here or simliar)

### Fast and slow

Before we go deeper into the topic, we can draw two conclusions:
- slow Python is not "slow": it is still blazingly fast on an absolute scale, e.g if you need to loop over a few hundred points, it's still nothing. But it can add up!
- Numpy is a factor of 300 faster for this case (and: better reabable!)

=> there is _no reason_ to ever add (numerical) arrays with a for loop in Python (except for numba jit)

As mentioned, TensorFlow is basically Numpy. Let's check that out

In [7]:
rnd1_np = np.random.uniform(size=10, low=0, high=10)
rnd1_np  # adding a return value on the last line without assigning it prints the value

array([0.71323006, 4.0400747 , 5.81932726, 3.45237458, 5.39951802,
       4.81768572, 8.58420365, 7.1927094 , 4.46526211, 1.19496194])

In [8]:
rnd1 = tf.random.uniform(shape=(10,),  # notice the "shape" argument: it's more picky than Numpy
                         minval=0,
                         maxval=10,
                         dtype=tf.float64)
rnd2 = tf.random.uniform(shape=(10,),
                         minval=0,
                         maxval=10,
                         dtype=tf.float64)

In [9]:
rnd1

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([6.70239689, 6.79552786, 2.59290637, 5.41634096, 5.08920428,
       1.85237492, 2.60647575, 8.87198757, 9.07753705, 9.59340374])>

This is in fact a "numpy array wrapped" and can explicitly be converted to an array

In [10]:
rnd1.numpy()

array([6.70239689, 6.79552786, 2.59290637, 5.41634096, 5.08920428,
       1.85237492, 2.60647575, 8.87198757, 9.07753705, 9.59340374])

Other operations act as we would expect it

In [11]:
rnd1 + 10

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([16.70239689, 16.79552786, 12.59290637, 15.41634096, 15.08920428,
       11.85237492, 12.60647575, 18.87198757, 19.07753705, 19.59340374])>

... and it converts itself (often) to Numpy when needed.

In [12]:
np.sqrt(rnd1)

array([2.58889878, 2.60682333, 1.6102504 , 2.32730337, 2.25592648,
       1.36101981, 1.61445835, 2.97858818, 3.01289513, 3.09732203])

We can slice it...

In [13]:
rnd1[1:3]

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([6.79552786, 2.59290637])>

...expand it....

In [14]:
rnd1[None, :, None]

<tf.Tensor: shape=(1, 10, 1), dtype=float64, numpy=
array([[[6.70239689],
        [6.79552786],
        [2.59290637],
        [5.41634096],
        [5.08920428],
        [1.85237492],
        [2.60647575],
        [8.87198757],
        [9.07753705],
        [9.59340374]]])>

...and broadcast with the known (maybe slightly stricter) rules

In [15]:
matrix1 = rnd1[None, :] * rnd1[:, None]

## Equivalent operations

Many operations that exist in Numpy also exist in TensorFlow, sometimes with a different name.

The concept however is exactly the same: we have higher level objects such as Tensors (or arrays) and call operations on it with arguments. This is a "strong limitation" (theoretically) of what we can do, however, since we do math, there is only a limited set we need, and in practice this suffices for 98% of the cases.

Therefore we won't dive too deep into the possibilities of TensorFlow/Numpy regarding operations but it is suggested to [read the API docs](https://www.tensorflow.org/versions), many are self-explanatory. It can be surprising that there is also some support for more exotic elements such as [RaggedTensors and operations](https://www.tensorflow.org/api_docs/python/tf/ragged?) and [SparseTensors and operations](https://www.tensorflow.org/api_docs/python/tf/sparse?)

Mostly, the differences and the terminology will be introduced.

In [16]:
tf.sqrt(rnd1)

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([2.58889878, 2.60682333, 1.6102504 , 2.32730337, 2.25592648,
       1.36101981, 1.61445835, 2.97858818, 3.01289513, 3.09732203])>

In [134]:
tf.reduce_sum(matrix1, axis=0)  # with the axis argument to specify over which to reduce

<tf.Tensor: shape=(10,), dtype=float64, numpy=
array([392.74809437, 398.20539735, 151.93953018, 317.38758917,
       298.2179833 , 108.54575323, 152.73467128, 519.8821064 ,
       531.92692658, 562.15576303])>

## What we can't do: assignements

The idea of TensorFlow evolves around building a Graph$^{1)}$, the Eager (Numpy-like) execution is more of a "we get it for free addition", but does not lead the development ideas. This has one profound implication, namely that we cannot make an _assignement_ to a Tensor, because it is a node in a graph. The logic just does not work (exception: `tf.Variable`$^{2)}). This does not mean that TensorFlow would not perform in-place operations _behind the scenes_ - it very well does if it is save to do so. Since TensorFlow knows the whole graph with all dependencies, this can be figured out.

Even if EagerMode, assignements could work (as for Numpy arrays), they are forbidden for consistency (one of the great plus points of TensorFlow).

### TensorFlow list

an advanced structure that can also be modified is the [`tf.TensorArray`](https://www.tensorflow.org/api_docs/python/tf/TensorArray?), which acts like a list or ArrayList (in other languages). It is however advanced and not usually used, so we won't go into the details here.

_1) if you're familiar with TensorFlow 1, this statement would suprise you as pretty obvious; but in TensorFlow 2, this is luckily more hidden.  
2) more general, `ResourceHandler`_

In [136]:
rnd1_np[5] = 42

In [138]:
try:
    rnd1[5] = 42
except TypeError as error:
    print(error)

'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment


### Speed comparison

Let's do the same calculation as with Numpy. The result should be comparable: both are AOT compiled libraries specialized on numerical, vectorized operations.

In [18]:
rnd1_big = tf.random.uniform(shape=(size1,),  # notice the "shape" argument: it's more picky than Numpy
                         minval=0,
                         maxval=10,
                         dtype=tf.float64)
rnd2_big = tf.random.uniform(shape=(size1,),
                         minval=0,
                         maxval=10,
                         dtype=tf.float64)

In [19]:
%%timeit
rnd1_big + rnd2_big

48.5 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Looks like the same as Numpy. Let's compare with smaller arrays

In [20]:
rnd1_np = rnd1.numpy()
rnd2_np = rnd2.numpy()

In [21]:
%%timeit
rnd1_np + rnd2_np

363 ns ± 3.21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
%%timeit
rnd1 + rnd2

8.61 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### TensorFlow is slow?

We see now a significant difference in the runtime! This is because TensorFlow has a larger overhead than Numpy. As seen before, this is not/barely noticable for larger arrays, however for very small calculations, this is visible.

There is more overhead because TensorFlow tries to be "smarter" about many things than Numpy and does not simply directly execute the computation.

The cost is a slowdown on very small operations but a better scaling and improved performance with larger arrays and more complicated calculations.

In [23]:
# size_big = 10  # numpy faster
size_big = 20000  # sameish
# size_big = 100000  # TF faster
# size_big = 1000000  # TF faster
# size_big = 10000000  # TF faster
# size_big = 100000000  # TF faster

In [24]:
%%timeit
tf.random.uniform(shape=(size_big,), dtype=tf.float64) + tf.random.uniform(shape=(size_big,), dtype=tf.float64)

239 µs ± 45.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%%timeit
np.random.uniform(size=(size_big,)) + np.random.uniform(size=(size_big,))

249 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## TensorFlow kernels

In general, TensorFlow is preciser in what input arguments are required compared to Numpy and does less automatic dtype casting and asks more explicit for shapes. For example, integers don't work in the logarithm. However, this error message illustrates very well the kernel dispatch system of TensorFlow, so lets do it!

In [26]:
try:
    tf.math.log(5)
except tf.errors.NotFoundError as error:
    print(error)

Could not find valid device for node.
Node:{{node Log}}
All kernels registered for op Log :
  device='XLA_GPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_CPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='XLA_GPU_JIT'; T in [DT_FLOAT, DT_DOUBLE, DT_COMPLEX64, DT_BFLOAT16, DT_COMPLEX128, DT_HALF]
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_HALF]
  device='GPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_COMPLEX128]
  device='CPU'; T in [DT_COMPLEX64]
  device='CPU'; T in [DT_BFLOAT16]
  device='CPU'; T in [DT_DOUBLE]
  device='CPU'; T in [DT_HALF]
  device='CPU'; T in [DT_FLOAT]
 [Op:Log]


What we see here: it searches the registered kernels and does not find any that supports this operation. We find different classifications:
- GPU: normal GPU kernel
- CPU: normal CPU kernel
- XLA: [Accelerated Linear Algebra](https://www.tensorflow.org/xla) is a high-level compiler that can fuse operations, which would result in single calls to a kernel, to a single kernel.

## tf.function

We now want to see the JIT in action. Therefore, we use the example from the slides and start modifying it.

In [27]:
def add_log(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

As seen before, we can use it like Python. To make sure that we know when the actual Python is executed, we inserted a print and a `tf.print`, the latter is a TensorFlow operation and therefore expected to be called everytime we compute something.

In [28]:
add_log(4., 5.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=2.9957323>

In [29]:
add_log(42., 52.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=7.6889133>

As we see, both the Python and TensorFlow operation execute. Now we can do the same with a decorator. Note that so far we entered pure Python numbers, not Tensors. Since we ran in eager mode, this did not matter so far.

In [30]:
@tf.function(autograph=True)
def add_log_tf(x, y):
    print('running Python')
    tf.print("running TensorFlow")
    x_sq = tf.math.log(x)
    y_sq = tf.math.log(y)
    return x_sq + y_sq

In [31]:
add_log_tf(1., 2.)

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=0.6931472>

In [32]:
add_log_tf(11., 21.)  # again with different numbers

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=5.442418>

As we see, Python is still run: this happens because 11. is not equal to 1., TensorFlow does not convert those to Tensors. Lets use it in the right way, with Tensors

In [33]:
add_log_tf(tf.constant(1.), tf.constant(2.))  # first compilation

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=0.6931472>

In [34]:
add_log_tf(tf.constant(11.), tf.constant(22.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=5.488938>

Now only the TensorFlow operations get executed! Everything else became static. We can illustrate this more extremely here

In [35]:
@tf.function(autograph=True)
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [36]:
add_rnd(tf.constant(1.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.51427203>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.91420484>)

The first time, the numpy code was executed as well, no difference so far. However, running it a second time, only the TensorFlow parts can change

In [37]:
add_rnd(tf.constant(1.))

running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.51427203>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.24060977>)

In [38]:
add_rnd(tf.constant(2.))

running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=1.0285441>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.770922>)

We see now clearly: TensorFlow executes the function but _only cares about the TensorFlow operations_ , everything else is regarded as static. This can be a large pitfall! If we would execute this function _without_ the decorator, we would get a different result, since Numpy is also sampling a new random variable every time.

### large functions

That having said, we can build graphs that require thousands of lines of Python code to stick them together correctly. Function calls in function calls etc are all possible.

### Shapes

Tensors have a shape, similar to Numpy arrays. But Tensors have two kind of shapes, a static and a dynamic shape. The static shape is what can be inferred _before_ executing the computation while the dynamic shape is only inferred during the execution of the code. The latter typically arises with random variables and masking or cuts.

We can access the static shape with `Tensor.shape`

If the shape is known inside a graph, this will be the same. If the shape is unknown, the unknown axis will be None.

In [129]:
@tf.function(autograph=False)
def func_shape(x):
    print(f"static shape: {x.shape}")  # static shape
    tf.print('dynamic shape ',tf.shape(x))  # dynamic shape
    x = x[x>3.5]
    print(f"static shape cuts applied: {x.shape}")  # static shape
    tf.print('dynamic shape cuts applied',tf.shape(x))  # dynamic shape

In [130]:
func_shape(rnd1)

static shape: (10,)
static shape cuts applied: (None,)
dynamic shape  [10]
dynamic shape cuts applied [7]


We can access the axes by indexing

In [131]:
rnd3 = rnd1[None, :] * rnd1[:, None]

In [132]:
tf.shape(rnd3)[1]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [133]:
rnd3.shape[1]

10

## Variables

TensorFlow offers the possibility to have statefull objects inside a compiled graph (which e.g. is not possible with Numba). The most commonly used one is the `tf.Variable`. Technically, they are automatically captured on the function compilation and belong to it.

In [40]:
var1 = tf.Variable(1.)

In [41]:
@tf.function(autograph=True)
def scale_by_var(x):
    print('running Python')
    tf.print("running TensorFlow")
    return x * var1

In [42]:
scale_by_var(tf.constant(1.))

running Python
running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [43]:
scale_by_var(tf.constant(2.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [44]:
var1.assign(42.)
scale_by_var(tf.constant(1.))

running TensorFlow


<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

As we see, the output changed. This is of course especially useful in the context of model fitting libraries, be it likelihoods or neural networks.

In [45]:
def add_rnd(x):
    print('running Python')
    tf.print("running TensorFlow")
    rnd_np = np.random.uniform()
    rnd_tf = tf.random.uniform(shape=())
    return x * rnd_np, x * rnd_tf

In [46]:
add_rnd(tf.constant(1.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=0.27471915>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.90596545>)

In [47]:
add_rnd(tf.constant(2.))

running Python
running TensorFlow


(<tf.Tensor: shape=(), dtype=float32, numpy=1.8032616>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.99052715>)

This means that we can use Numpy fully compatible in eager mode, but not when decorated.

In [48]:
def try_np_sqrt(x):
    return np.sqrt(x)

In [49]:
try_np_sqrt(tf.constant(5.))

2.236068

In [50]:
try_np_sqrt_tf = tf.function(try_np_sqrt, autograph=False)  # equivalent to decorator

In [51]:
try:
    try_np_sqrt_tf(tf.constant(5.))
except NotImplementedError as error:
    print(error)

Cannot convert a symbolic Tensor (x:0) to a numpy array.


As we see, Numpy complains in the graph mode, given that it cannot handle the Symbolic Tensor.

Having the `tf.function` decorator means that we can't use any Python dynamicity. What fails when decorated but works nicely if not:

In [52]:
def greater_python(x, y):
    if x > y:
        return True
    else:
        return False

In [53]:
greater_python(tf.constant(1.), tf.constant(2.))

False

This works again, and will fail with the graph decorator.

In [54]:
greater_python_tf = tf.function(greater_python, autograph=False)

In [55]:
try:
    greater_python_tf(tf.constant(1.), tf.constant(2.))
except Exception as error:
    print(error)

using a `tf.Tensor` as a Python `bool` is not allowed: AutoGraph is disabled in this function. Try decorating it directly with @tf.function.


The error message hints at something: while this does not work now - Python does not yet now the value of the Tensors so it can't decide whether it will evaluate to True or False - there is the possibility of "autograph": it automatically converts (a subset) of Python to TensorFlow: while loops, for loops through Tensors and conditionals. However, this is usually less effective and more errorprone than using explicitly the `tf.*` functions. Lets try it!

In [56]:
greater_python_tf_autograph = tf.function(greater_python, autograph=True)

In [57]:
greater_python_tf_autograph(tf.constant(1.), tf.constant(2.))

<tf.Tensor: shape=(), dtype=bool, numpy=False>

This now works neatless! But we're never sure.

To do it explicitly, we can do that as well.

In [58]:
code = tf.autograph.to_code(greater_python)
print(code)

def tf__greater_python(x, y):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('greater_python', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:

        def get_state():
            return ()

        def set_state(loop_vars):
            pass

        def if_true():
            try:
                do_return = True
                retval_ = fscope.mark_return_value(True)
            except:
                do_return = False
                raise
            return (do_return, retval_)

        def if_false():
            try:
                do_return = True
                retval_ = fscope.mark_return_value(False)
            except:
                do_return = False
                raise
            return (do_return, retval_)
        cond = (x > y)
        (do_return, retval_) = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('do_return',

## Performance

In the end, this is what matters. And a comparison would be nice. Let's do that and see how Numpy and TensorFlow compare.

In [59]:
nevents = 10000000
data_tf = tf.random.uniform(shape=(nevents,), dtype=tf.float64)
data_np = np.random.uniform(size=(nevents,))

In [60]:
def calc_np(x):
    x_init = x
    i = 42.
    x = np.sqrt(np.abs(x_init * (i + 1.)))
    x = np.cos(x - 0.3)
    x = np.power(x, i + 1)
    x = np.sinh(x + 0.4)
    x = x ** 2
    x = x / np.mean(x)
    x = np.abs(x)
    logx = np.log(x)
    x = np.mean(logx)
    
    x1 = np.sqrt(np.abs(x_init * (i + 1.)))
    x1 = np.cos(x1 - 0.3)
    x1 = np.power(x1, i + 1)
    x1 = np.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / np.mean(x1)
    x1 = np.abs(x1)
    logx = np.log(x1)
    x1 = np.mean(logx)
    
    x2 = np.sqrt(np.abs(x_init * (i + 1.)))
    x2 = np.cos(x2 - 0.3)
    x2 = np.power(x2, i + 1)
    x2 = np.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / np.mean(x2)
    x2 = np.abs(x2)
    logx = np.log(x2)
    x2 = np.mean(logx)
    return x + x1 + x2

calc_np_numba = numba.jit(nopython=True, parallel=True)(calc_np)

In [61]:
def calc_tf(x):
    x_init = x
    i = 42.
    x = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x = tf.cos(x - 0.3)
    x = tf.pow(x, tf.cast(i + 1, tf.float64))
    x = tf.sinh(x + 0.4)
    x = x ** 2
    x = x / tf.reduce_mean(x)
    x = tf.abs(x)
    x = tf.reduce_mean(tf.math.log(x))
    
    x1 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x1 = tf.cos(x1 - 0.3)
    x1 = tf.pow(x1, tf.cast(i + 1, tf.float64))
    x1 = tf.sinh(x1 + 0.4)
    x1 = x1 ** 2
    x1 = x1 / tf.reduce_mean(x1)
    x1 = tf.abs(x1)
    
    x2 = tf.sqrt(tf.abs(x_init * (tf.cast(i, dtype=tf.float64) + 1.)))
    x2 = tf.cos(x2 - 0.3)
    x2 = tf.pow(x2, tf.cast(i + 1, tf.float64))
    x2 = tf.sinh(x2 + 0.4)
    x2 = x2 ** 2
    x2 = x2 / tf.reduce_mean(x2)
    x2 = tf.abs(x2)
    
    return x + x1 + x2

calc_tf_func = tf.function(calc_tf, autograph=False)

In [62]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_tf_func(data_tf)

192 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [63]:
%%timeit -n1 -r1  # compile time, just for curiosity
calc_np_numba(data_np)

2.79 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [64]:
%timeit calc_np(data_np)  # not compiled

2.01 s ± 23.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%timeit calc_tf(data_tf)  # not compiled

721 ms ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
%%timeit -n1 -r7
calc_np_numba(data_np)

400 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit -n1 -r7
calc_tf_func(data_tf)

181 ms ± 9.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


We can now play around with this numbers. Depending on the size (we can go up to 10 mio) and parallelizability of the problem, the numbers differ..

In general:
- Numpy is faster for small numbers
- TensorFlow is faster for larger arrays and well parallelizable computations. Due to the larger overhead in dispatching in eager mode, it is significantly slower for very small (1-10) sample sizes.

=> there is no free lunch

Note: this has not run on a GPU, which would automatically happen for TensorFlow.

In [184]:
def calc_tf2(x, n):
    sum_init = tf.zeros_like(x)
    for i in range(1, n + 1):
        x = tf.sqrt(tf.abs(x * (tf.cast(i, dtype=tf.float64) + 1.)))
        x = tf.cos(x - 0.3)
        x = tf.pow(x, tf.cast(i + 1, tf.float64))
        x = tf.sinh(x + 0.4)
        x = x ** 2
        x = x / tf.reduce_mean(x, axis=None)
        x = tf.abs(x)
        x = x - tf.reduce_mean(tf.math.log(x, name="Jonas_log"), name="Jonas_mean")  # name for ops, see later ;)
        sum_init += x
    return sum_init

calc_tf_func2 = tf.function(calc_tf2, autograph=False)

@numba.njit(parallel=True)  # njit is equal to jit(nopython=True), meaning "compile everything or raise error"
def calc_numba2(x, n):
    sum_init = np.zeros_like(x)
    for i in range(1, n + 1):
        x = np.sqrt(np.abs(x * (i + 1.)))
        x = np.cos(x - 0.3)
        x = np.power(x, i + 1)
        x = np.sinh(x + 0.4)
        x = x ** 2
        x = x / np.mean(x)
        x = np.abs(x)
        x = x - np.mean(np.log(x))
        sum_init += x
    return sum_init

In [69]:
%%timeit -n1 -r1  #compile
calc_numba2(rnd1_big.numpy(), 1)

936 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
calc_numba2(rnd1_big.numpy(), 1)

array([2.44106763, 1.57559678, 1.41465653, ..., 1.06970403, 0.8020929 ,
       0.69118826])

In [71]:
%%timeit -n1 -r1  #compile
calc_tf_func2(rnd1_big, 1)

20.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [72]:
calc_tf_func2(rnd1_big, 1)

<tf.Tensor: shape=(100000,), dtype=float64, numpy=
array([2.44106763, 1.57559678, 1.41465653, ..., 1.06970403, 0.8020929 ,
       0.69118826])>

In [73]:
%%timeit
calc_numba2(rnd1_big.numpy(), 1)

2.08 ms ± 74.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [74]:
%%timeit
calc_tf_func2(rnd1_big, 1)

1.37 ms ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
calc_tf_func2(rnd1_big, 10)

<tf.Tensor: shape=(100000,), dtype=float64, numpy=
array([13.92446397, 15.40159755, 12.99920382, ..., 16.28577591,
       13.5781476 , 24.10426561])>

In [76]:
calc_numba2(rnd1_big.numpy(), 10)

array([13.92446397, 15.40159755, 12.99920382, ..., 16.28577591,
       13.5781476 , 24.10426561])

In [77]:
%%timeit
calc_numba2(rnd1_big.numpy(), 10)

20.1 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
%%timeit
calc_tf_func2(rnd1_big, 10)

16.9 ms ± 737 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Control flow

While TensorFlow is independent of the Python control flow, it has its own functions for that, mainly:
- while_loop(): a while loop taking a body and condition function
- cond(): if-like
- case and switch_case: if/elif statements
- tf.where

In [79]:
def true_fn():
    return 1.

def false_fn():
    return 0.

value = tf.cond(tf.greater(111., 42.), true_fn=true_fn, false_fn=false_fn)

In [80]:
value

1.0

### While loops

We can create while loops in order to have some kind of repetitive task

In [81]:
def cond(x, y):
    return x > y

def body(x, y):
    return x / 2, y + 1

x, y = tf.while_loop(cond=cond,
                     body=body,
                     loop_vars=[100., 1.])

In [82]:
x, y

(<tf.Tensor: shape=(), dtype=float32, numpy=3.125>,
 <tf.Tensor: shape=(), dtype=float32, numpy=6.0>)

### map a function

We can also map a function on each element. While this is not very efficient, it allows for high flexibility.

In [83]:
%%timeit -n1 -r1
tf.map_fn(tf.math.sin, rnd1_big)

10.8 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [84]:
%%timeit -n1 -r1
tf.vectorized_map(tf.math.sin, rnd1_big)  # can speedup things sometimes

52.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [85]:
@tf.function(autograph=False)
def do_map(func, tensor):
    return tf.map_fn(func, tensor)

do_map(tf.math.sin, rnd1_big)

<tf.Tensor: shape=(100000,), dtype=float64, numpy=
array([-0.65656248, -0.91235429,  0.97561133, ...,  0.51088504,
       -0.00672329,  0.15346295])>

In [86]:
%%timeit -n1
do_map(tf.math.sin, rnd1_big)

860 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
%%timeit
tf.math.sin(rnd1_big)

459 µs ± 26.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This mapping is surely not optimal. However, it works.

## Gradients

TensorFlow allows us to calculate the automatic gradients.

In [88]:
var2 = tf.Variable(2.)

In [89]:
with tf.GradientTape() as tape:
    tape.watch(var2)  # actually watches all variables already by default
    y = var2 ** 3
y

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [90]:
grad = tape.gradient(y, var2)
grad

<tf.Tensor: shape=(), dtype=float32, numpy=12.0>

## Tasks to try out

Following are a few ideas to check whether you understand things

 - can you get the second derivative? play around ;)
 - write a shape that you know as a function
 - create the MC integral of it over a certain range

This allows to do many things with gradients and e.g. solve differential equations.

## Bonus: the graph

We talked about the graph back and forth, but _where is it_ ?

The graph can be retained from a function that was already traced.

In [176]:
concrete_func = calc_tf_func2.get_concrete_function(rnd1, 2)
concrete_func

In [177]:
graph = concrete_func.graph
graph

In [178]:
ops = graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'Cast/x' type=Const>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'mul' type=Mul>,
 <tf.Operation 'Abs' type=Abs>,
 <tf.Operation 'Sqrt' type=Sqrt>,
 <tf.Operation 'sub/y' type=Const>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'Cos' type=Cos>,
 <tf.Operation 'Cast_1/x' type=Const>,
 <tf.Operation 'Cast_1' type=Cast>,
 <tf.Operation 'Pow' type=Pow>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'Sinh' type=Sinh>,
 <tf.Operation 'pow_1/y' type=Const>,
 <tf.Operation 'pow_1' type=Pow>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'Mean' type=Mean>,
 <tf.Operation 'truediv' type=RealDiv>,
 <tf.Operation 'Abs_1' type=Abs>,
 <tf.Operation 'Jonas_log' type=Log>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'Jonas_mean' type=Mean>,
 <tf.Operation 'sub_1' type=Sub>,
 

In [179]:
log_op = ops[-6]
log_op

<tf.Operation 'Jonas_log_1' type=Log>

In [180]:
log_op.outputs

[<tf.Tensor 'Jonas_log_1:0' shape=(10,) dtype=float64>]

In [181]:
op_inputs_mean = ops[-4].inputs
op_inputs_mean

(<tf.Tensor 'Jonas_log_1:0' shape=(10,) dtype=float64>,
 <tf.Tensor 'Const_3:0' shape=(1,) dtype=int32>)

In [182]:
log_op.outputs[0] is op_inputs_mean[0]

True

The output of the log operation is the input to the mean operation! We can just walk along the graph here. TensorFlow Graphs are no magic, they are simple object that store their input, their output, their operation. That's it!